In [1]:
""" 

This section cleans up the parsed data and stores it as tables in a Postgresql database.
As the data is still in JSON format, some further parsing will have to be done before it can pass into SQL tables 

"""


' \n\nThis section cleans up the parsed data and stores it as tables in a Postgresql database.\nAs the data is still in JSON format, some further parsing will have to be done before it can pass into SQL tables \n\n'

In [3]:
import pandas as pd
from pandas.io import sql
import json
import regex as re
import psycopg2
from sqlalchemy import create_engine
import numpy as np

In [4]:
# Open the itunes dataset from data folder #

with open('./data/itunes_data.json', 'r') as file:
    d=json.load(file)
    file.close()

itunes_json=d

In [12]:
# Use pandas json parser to export to pandas dataframe object #

itunes_main=pd.read_json('./data/itunes_data.json',orient='records')

In [14]:
# Sub-setting the itunes data, and renaming to standardise ID keys across the different tables #
# At this stage, we will be a bit "greedy" with the features and store more variables for a start # 

itunes_orig_features=['trackId','wrapperType','trackName','artistName',\
                      'trackRentalPrice','trackPrice','trackHdPrice',\
                      'trackHdRentalPrice','releaseDate','country','currency','longDescription']


itunes=itunes_main[itunes_orig_features]


itunes_features=['iTunes_ID','wrapperType','movie_title','artist_name',\
                 'rental_price','main_price','hd_main_price',\
                 'hd_rental_price','releaseDate','country','currency','synopsis']

# Rename the itunes dataframe variables # 
itunes.columns=itunes_features



In [15]:
itunes.head()

,iTunes_ID,wrapperType,movie_title,artist_name,rental_price,b'main_price',hd_main_price,hd_rental_price,releaseDate,country,currency,synopsis
0,1.434879e+09,track,The Predator,Shane Black,NaN,19.99,19.99,NaN,2018-09-14T07:00:00Z,USA,USD,From the outer reaches of space to the to the ...
1,1.406516e+09,track,Mission: Impossible - Fallout,Christopher McQuarrie,5.99,14.99,19.99,5.99,2018-07-27T07:00:00Z,USA,USD,On a dangerous assignment to recover stolen pl...
2,1.417714e+09,track,The Meg,Jon Turteltaub,5.99,19.99,19.99,5.99,2018-08-10T07:00:00Z,USA,USD,A deep-sea submersible filled with an internat...
3,1.404080e+09,track,Mile 22,Peter Berg,5.99,12.99,14.99,5.99,2018-08-17T07:00:00Z,USA,USD,In a visceral modern thriller from the directo...
4,1.392900e+09,track,Incredibles 2,Brad Bird,4.99,14.99,14.99,5.99,2018-06-15T07:00:00Z,USA,USD,"In Incredibles 2, Helen is called on to lead a..."


In [4]:
# Subsetting the OMdb search data #
# This table is important as it contains both IMdb ID and the iTunes track ID # 
# which will be used to join the IMdb data and the iTunes data # 

OMdb_search_dat_df=pd.read_json('OMdb_search_dat.json',orient='columns')
OMdb_search_dat_df.columns= ['IMdb_ID', 'TMdb_ID', 'iTunes_ID', 'search_strs', 'titles', 'years']

In [3]:
OMdb_results_df=pd.read_json('./data/OMdb_data.json',orient='records')

In [17]:
sum(OMdb_results_df['BoxOffice'].apply(lambda x: np.nan if x in ['N/A', 'NaN'] else x).isnull())

236

In [10]:
with open ('./data/OMdb_data.json','r') as file:
    d=json.load(file)
    file.close()
OMdb_dat=d

In [12]:
OMdb_rat_open=unpack_ratings_OMdb(OMdb_dat)

In [14]:
OMdb_unpacked=pop_actors_and_directors(OMdb_rat_open)

In [15]:
with open ('./data/OMdb_unpacked.json','w') as outfile:
    json.dump(OMdb_unpacked,outfile)

In [16]:
OMdb_unpacked_df=pd.read_json('./data/OMdb_unpacked.json',orient='records')

In [17]:
OMdb_orig_features=['Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director',\
       'Episode', 'Genre', 'IMdb_score', 'Language', 'Metacritic_score',\
       'Metascore', 'Plot', 'Poster', 'Production', 'RT_score', 'Rated',\
       'Released', 'Runtime', 'Title', 'Type', 'Website',\
        'Year', 'actor_1', 'actor_2', 'director_1', 'director_2',\
       'imdbID', 'imdbRating', 'imdbVotes', 'num_actor',\
       'num_director']
OMdb_df=OMdb_unpacked_df[OMdb_orig_features]
OMdb_features=['Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director',\
       'Episode', 'Genre', 'IMdb_score', 'Language', 'Metacritic_score',\
       'Metascore', 'Plot', 'Poster', 'Production', 'RT_score', 'Rated',\
       'Released', 'Runtime', 'Title', 'Type', 'Website',\
        'Year', 'actor_1', 'actor_2', 'director_1', 'director_2',\
       'IMdb_ID', 'imdbRating', 'imdbVotes', 'num_actor', \
       'num_director']
OMdb_df.columns=OMdb_features


In [45]:
## Create tables in an existing PostgreSQL db called "movies" ## 

connection=psycopg2.connect(host="localhost",database="movies", user="postgres", password="postgres")

In [160]:

engine = create_engine('postgresql://localhost:5432/movies')
itunes_df.to_sql('itunes',engine,if_exists='replace',index=False)
OMdb_search_dat_df.to_sql('omdb_search',engine,if_exists='replace',index=False)
OMdb_df.to_sql('omdb',engine,if_exists='replace',index=False)

In [48]:
query="""
    SELECT column_name,data_type
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME= 'omdb'
"""

pd.read_sql(query,connection)

,column_name,data_type
0,Actors,text
1,Awards,text
2,BoxOffice,text
3,Country,text
4,DVD,text
5,Director,text
6,Episode,double precision
7,Genre,text
8,IMdb_score,double precision
9,Language,text


In [79]:
query="""
    SELECT ""
    FROM omdb
    LIMIT 5
"""

pd.read_sql(query,connection)

,Actors
0,"Janina Kuzma, Sandra Lahnsteiner, Julia Mancus..."
1,N/A
2,N/A
3,"Travis Rice, Mark Landvik, John Jackson, Nicol..."
4,"Iris Berard-Malenaucka, Iris Bernard-Malenauck..."


In [361]:
query="""
    SELECT *
    FROM itunes
    LIMIT 5
"""

pd.read_sql(query,connection)

,iTunes_ID,wrapperType,movie_title,artist_name,rental_price,main_price,hd_main_price,hd_rental_price,releaseDate,country,currency,synopsis
0,1.434879e+09,track,The Predator,Shane Black,NaN,19.99,19.99,NaN,2018-09-14T07:00:00Z,USA,USD,From the outer reaches of space to the to the ...
1,1.406516e+09,track,Mission: Impossible - Fallout,Christopher McQuarrie,NaN,14.99,19.99,NaN,2018-07-27T07:00:00Z,USA,USD,On a dangerous assignment to recover stolen pl...
2,1.417714e+09,track,The Meg,Jon Turteltaub,5.99,12.99,12.99,5.99,2018-08-10T07:00:00Z,USA,USD,A deep-sea submersible filled with an internat...
3,1.404080e+09,track,Mile 22,Peter Berg,5.99,12.99,14.99,5.99,2018-08-17T07:00:00Z,USA,USD,In a visceral modern thriller from the directo...
4,1.392900e+09,track,Incredibles 2,Brad Bird,4.99,14.99,19.99,5.99,2018-06-15T07:00:00Z,USA,USD,"In Incredibles 2, Helen is called on to lead a..."


In [82]:
query="""
    SELECT *
    FROM omdb
    LEFT JOIN omdb_search
    ON omdb."IMdb_ID"=omdb_search."IMdb_id"
"""

omdb_big=pd.read_sql(query,connection)

In [84]:
omdb_big.to_sql('omdb_big',con=engine,if_exists='replace')

In [95]:
query="""
    SELECT iTunes_ID
    FROM omdb_big
    LIMIT 5
"""

pd.read_sql(query,connection)

,itunes_id
0,1179287911
1,767303057
2,716284928
3,456513023
4,478793486


In [97]:
query="""
    SELECT *
    FROM itunes
    LEFT JOIN omdb_big
    ON itunes."iTunes_ID"=omdb_big.iTunes_ID
    
"""

main=pd.read_sql(query,connection)

In [99]:
main.to_sql('main',con=engine,if_exists='replace',index=False)

In [134]:
query="""
    SELECT COUNT("IMdb_id"), movie_title
    FROM main
    GROUP BY movie_title
    ORDER BY COUNT("IMdb_id") DESC
    
"""

pd.read_sql(query,connection)

DatabaseError: Execution failed on sql '
    SELECT COUNT("IMdb_id"), movie_title
    FROM main
    WHERE COUNT("IMdb_id") >1
    GROUP BY movie_title
    ORDER BY COUNT("IMdb_id") DESC
    
': aggregate functions are not allowed in WHERE
LINE 4:     WHERE COUNT("IMdb_id") >1
                  ^
